In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing the Dependencies

In [ ]:
! pip install -r /content/drive/MyDrive/Llama3/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
import json
import torch
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline)

**HF account Configuration**

In [ ]:
config_data = json.load(open("/content/drive/MyDrive/Llama3/config.json"))
HF_TOKEN = config_data["HF_TOKEN"]


In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B"

**Quantisation Configuration**

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

**Loading the Tokenizer and the LLM**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          token=HF_TOKEN)

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    token=HF_TOKEN
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
text_generator = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    max_new_tokens = 128

)

In [ ]:
def get_response(prompt):
  sequences = text_generator(prompt)
  gen_text = sequences[0]["generated_text"]
  return gen_text

In [ ]:
prompt = "What is ML?"

In [ ]:
llama3_response = get_response(prompt)

In [ ]:
print(llama3_response[len(prompt):])

## Finetune with papers

In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q joblib
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 74.1 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
!pip install -q llama-index
!pip install -q llama-index-llms-huggingface
!pip install -q llama-index-embeddings-langchain
!pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.4 MB/s eta 0:00:00


In [ ]:
# import os, logging, sys

# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## HF login

In [ ]:
# from huggingface_hub import login

# os.environ["HF_KEY"] = "" # add your own Huggging Face access token
# login(token=os.environ.get('HF_KEY'),add_to_git_credential=True)

## Retrieve

In [ ]:
from llama_index.core import SimpleDirectoryReader
import psutil
import os

# Specify the directory containing the papers
papers_directory = "/content/drive/MyDrive/LLMFTnRAG_202404/papers_temp2"

# Function to find all PDF files in a directory and its subdirectories
def find_all_pdfs(directory):
    pdf_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.pdf'):
                pdf_files.append(os.path.join(root, file))
                # print(f"Found PDF file: {os.path.join(root, file)}")
    return pdf_files

# Get a list of all PDF files in the directory and its subdirectories
pdf_files = find_all_pdfs(papers_directory)

# Initialize the documents list
documents = []

# Function to check memory usage
def check_memory_usage(threshold=80):
    memory = psutil.virtual_memory()
    return memory.percent < threshold


# Batch size for processing PDF files
batch_size = 10

# Process PDF files in batches
for i in range(0, len(pdf_files), batch_size):
    if not check_memory_usage():
        logging.warning("Memory usage is high, pausing processing.")
        break
    batch = pdf_files[i:i+batch_size]
    for pdf_file in batch:
        try:
            reader = SimpleDirectoryReader(input_dir=os.path.dirname(pdf_file), required_exts=".pdf").load_data()
            documents.extend(reader)
        except Exception as e:
            logging.warning(f"Failed to read {pdf_file}: {e}")

# Print the first document
if documents:
    print("████████████████████████████████████████████████████████████████████████████████████████████████")
    print(documents[0])
else:
    print("No documents found.")

████████████████████████████████████████████████████████████████████████████████████████████████
Doc ID: 4342b6fd-cf95-4698-b8a7-3a47d54e9d17
Text: Instructions for Use Infinity ® CentralStation ™ with Infinity ®
Telemetry System ™ and with Infinity ® M300 ™ WARNING: For a full
understanding of the  performance characteristics of  this device, the
user should carefully read this manual  before use of the device.


## Vector Embedding

In [16]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

EMBEDDING_MODEL_NAME = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"

embed_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

<ipython-input-16-bff40edde72c>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embed_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Init the Vector Store Index

In [17]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, embed_model = embed_model)

## Augment

In [20]:
# Prompt settings

#from llama_index.core.prompts.prompts import PromptTemplate
from llama_index.core import PromptTemplate

system_prompt = """<|SYSTEM|># You are an AI-enabled admin assistant.
Your goal is to answer questions accurately using only the context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

# LLM_MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf" # modify
LLM_MODEL_NAME = "meta-llama/Meta-Llama-3-8B"

In [ ]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM

# To import models from HuggingFace directly
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=512,
    generate_kwargs={"temperature": 0.1, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=LLM_MODEL_NAME,
    model_name=LLM_MODEL_NAME,
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

# Migrating from serviceContext to Settings

In [22]:
from llama_index.core import Settings

Settings.embed_model = embed_model
Settings.llm = llm
Settings.chunk_size = 1024
#Settings.chunk_overlap = 256

In [ ]:
import os
import joblib, pickle
import faiss
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# Function to save components
def save_model_components(index, embed_model, hf_llm, path):
    os.makedirs(path, exist_ok=True)

    # Save the FAISS index
    # index.save(os.path.join(path, 'vector_store_index.faiss'))
    # faiss_index = index  # Adjust this according to your actual class structure
    # index_path = os.path.join(path, 'vector_store_index.faiss')
    # faiss.write_index(faiss_index, index_path)

    # Save the embedding model
    # embed_model_path = os.path.join(path, 'embedding_model3.pkl')
    # with open(embedding_model_path, "wb") as f:
    #     pickle.dump(embed_model, f)
    # Define the embedding model name
    EMBEDDING_MODEL_NAME = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"

    # Load the embedding model
    embed_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)


    # Move the underlying HuggingFace model to CPU
    embed_model.client = SentenceTransformer(EMBEDDING_MODEL_NAME, device='cpu')

    # Save the embedding model to a file using pickle
    embedding_model_path = "./embedding_model_cpu.pkl"
    with open(embedding_model_path, "wb") as f:
        pickle.dump(embed_model, f)

    model = AutoModelForCausalLM.from_pretrained(LLM_MODEL_NAME, max_memory=4.0)
    tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME)
    model.save_pretrained(os.path.join(path, 'llm_model'))
    tokenizer.save_pretrained(os.path.join(path, 'llm_tokenizer'))

    #Save the LLM model and tokenizer
    hf_llm.save_pretrained(os.path.join(path, 'llm_model'))
    hf_llm.save_model(os.path.join(path, 'llm_model'))

    llm.tokenizer.save_pretrained(os.path.join(path, 'llm_model'))
    joblib.dump(hf_llm, os.path.join(save_path, 'model.pkl'))
    joblib.dump(hf_llm.tokenizer, os.path.join(save_path, 'tokenizer.pkl'))

    #Save the settings
    settings_path = os.path.join(path, 'settings.pkl')
    with open(settings_path, 'wb') as f:
        joblib.dump(Settings, f)

# Save the model components
save_path = "llama_3_model_ft"
save_model_components(index, embed_model, llm, save_path)

## Generate

In [24]:
# Query engine initialization
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)